In [1]:
import pandas as pd
import plotly.express as px

In [2]:
cars_df = pd.read_csv('data/car_data.csv')

In [3]:
cars_df.drop([
        "engine_fuel",
        "feature_0",
        "feature_1",
        "feature_2",
        "feature_3",
        "feature_4",
        "feature_5",
        "feature_6",
        "feature_7",
        "feature_8",
        "feature_9"
    ], axis=1, inplace=True)

In [4]:
cars_df["location_region"].replace({
        "Минская обл.":"Minsk Region",
        "Гомельская обл.":"Gomel Region",
        "Витебская обл.":"Vitebsk Region",
        "Брестская обл.":"Brest Region",
        "Могилевская обл.":"Mogilev Region",
        "Гродненская обл.":"Hrodna Region"
    }, inplace=True)


In [5]:
cars_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38531 entries, 0 to 38530
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   manufacturer_name  38531 non-null  object 
 1   model_name         38531 non-null  object 
 2   transmission       38531 non-null  object 
 3   color              38531 non-null  object 
 4   odometer_value     38531 non-null  int64  
 5   year_produced      38531 non-null  int64  
 6   engine_has_gas     38531 non-null  bool   
 7   engine_type        38531 non-null  object 
 8   engine_capacity    38521 non-null  float64
 9   body_type          38531 non-null  object 
 10  has_warranty       38531 non-null  bool   
 11  state              38531 non-null  object 
 12  drivetrain         38531 non-null  object 
 13  price_usd          38531 non-null  float64
 14  is_exchangeable    38531 non-null  bool   
 15  location_region    38531 non-null  object 
 16  number_of_photos   385

In [6]:
cars_df["model_name"].value_counts()

Passat    1423
Astra      751
Golf       707
A6         687
Mondeo     637
          ... 
C1500        1
Alero        1
Impala       1
360          1
Aspen        1
Name: model_name, Length: 1118, dtype: int64

# Does the price affect the duration listed?



In [7]:
#fig = px.scatter(cars_df, x="price_usd", y="duration_listed", color="has_warranty")
#fig.update_layout(height = 500,width = 1500)
#fig.show()

# Does the manufacturer name affect the price?



In [10]:
x = cars_df['manufacturer_name'].value_counts()

y=[]
for name in x.index:
    y.append(cars_df[cars_df["manufacturer_name"] == name]['price_usd'].mean())

z={
    "Manufacturer Name":x.index,
    "Average Price":y
}
avgPrice_df = pd.DataFrame(data=z)

In [11]:
avgPrice_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Manufacturer Name  55 non-null     object 
 1   Average Price      55 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1008.0+ bytes


In [12]:
avgPrice_df.sort_values(by="Average Price", ascending=True, inplace=True)

In [28]:
box_manufacturer = px.bar(avgPrice_df, x="Manufacturer Name", y="Average Price")
box_manufacturer.update_layout(
    width=1200
)

box_manufacturer.show()

In [103]:
price_range = []
for price in cars_df["price_usd"]:
    if price <= 10000:
        price_range.append("0-10k")
    elif price <=20000:
        price_range.append("10k-20k")
    elif price <=30000:
        price_range.append("20k-30k")
    elif price <=40000:
        price_range.append("30k-40k")
    else:
        price_range.append("40k-50k")

cars_df["price_range"] = price_range

In [109]:
hist_duration = px.histogram(cars_df, x="price_range", y="duration_listed")
hist_duration.show()

# Do factors that don't affect the car's performance affect the price?



In [138]:
box_color = px.box(cars_df, x="color", y="price_usd",
                labels={
                    "color":"Color",
                    "price_usd":"Price (USD)"
                },
                  title="Color of car vs. Price")
box_color.show()

In [17]:
locs = cars_df["location_region"].value_counts()
locx = locs.index
locy = []
for location in locx:
    mean = cars_df[cars_df["location_region"] == location]["price_usd"].mean()
    locy.append(mean)

In [40]:
hist_location = px.histogram(x=locx, y=locy,
                            labels={
                                "x":"Regions"
                            },
                             title="---")
hist_location.update_layout(height=500, width=750, yaxis_title="Average Price")
hist_location.show()

In [48]:
cars_df["location_region"].value_counts()[:]

Minsk Region      24230
Gomel Region       3140
Vitebsk Region     3007
Brest Region       2991
Mogilev Region     2678
Hrodna Region      2485
Name: location_region, dtype: int64

In [50]:
hist_location = px.histogram(x=locx, y=locs,
                            labels={
                                "x":"Regions"
                            },
                             title="Number of Cars ")
hist_location.update_layout(height=500, width=750, yaxis_title="Number of Cars")
hist_location.show()

In [24]:
# x=cars_df["location_region"].value_counts()

Minsk Region      24230
Gomel Region       3140
Vitebsk Region     3007
Brest Region       2991
Mogilev Region     2678
Hrodna Region      2485
Name: location_region, dtype: int64

In [116]:
percent_df = pd.DataFrame(columns=["Manufacturer Name", "Minsk Region", "Gomel Region", "Vitebsk Region", "Brest Region", "Mogilev Region", "Hrodna Region"])
# percent_df = pd.DataFrame(columns=avgPrice_df["Manufacturer Name"])

maxPrice_df = avgPrice_df["Manufacturer Name"].tail(10)

for i in range(10):
    car = maxPrice_df.iloc[i]
    loclist = [car]
    j = 0
    for loc in locx:
        num = len(cars_df[(cars_df["manufacturer_name"] == car) & (cars_df["location_region"] == loc)].index)
        num /= locs.iloc[j]
        num *= 100
        loclist.append(num)
        j += 1
    percent_df.loc[i] = loclist

percent_df.head()
# j = 0
# for loc in locx:
#     loclist = []
#     for i in range(len(avgPrice_df.index)):
#         car = avgPrice_df["Manufacturer Name"].iloc[i]
#         num = len(cars_df[(cars_df["manufacturer_name"] == car) & (cars_df["location_region"] == loc)].index)
#         num /= locs.iloc[j]
#         num *= 100
#         loclist.append(num)
#     j += 1
#     percent_df.loc[loc] = loclist

,Manufacturer Name,Minsk Region,Gomel Region,Vitebsk Region,Brest Region,Mogilev Region,Hrodna Region
0,Cadillac,0.107305,0.127389,0.066511,0.033434,0.186706,0.201207
1,Acura,0.235246,0.031847,0.066511,0.133735,0.037341,0.040241
2,Buick,0.165085,0.031847,0.033256,0.167168,0.000000,0.000000
3,Mini,0.231118,0.191083,0.099767,0.033434,0.074683,0.000000
4,Infiniti,0.466364,0.445860,0.299302,0.300903,0.336072,0.321932


In [84]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [121]:
percent_df.head(5)

,Manufacturer Name,Minsk Region,Gomel Region,Vitebsk Region,Brest Region,Mogilev Region,Hrodna Region
0,Cadillac,0.107305,0.127389,0.066511,0.033434,0.186706,0.201207
1,Acura,0.235246,0.031847,0.066511,0.133735,0.037341,0.040241
2,Buick,0.165085,0.031847,0.033256,0.167168,0.000000,0.000000
3,Mini,0.231118,0.191083,0.099767,0.033434,0.074683,0.000000
4,Infiniti,0.466364,0.445860,0.299302,0.300903,0.336072,0.321932


In [137]:
car_colors = {'Cadillac':'red', 'Acura':'yellow', 'Buick':'blue', 'Mini':'orange', 'Infiniti':'green', 'Skoda':'purple', 'Land Rover':'grey',
       'Lexus':'pink', 'Jaguar':'white', 'Porsche':'black'}

fig_carlocs = make_subplots(rows=3, cols=2,
                           subplot_titles=("Minsk Region", "Gomel Region", "Vitebsk Region", "Brest Region", "Mogilev Region", "Hrodna Region"),)

def mapColors():
    colors = []
    for i in range(10):
        colors.append(car_colors[percent_df["Manufacturer Name"].iloc[i]])
    return colors

trace0 = go.Bar(x=percent_df["Minsk Region"], marker=dict(color = mapColors()), text=percent_df["Manufacturer Name"])
trace1 = go.Bar(x=percent_df["Gomel Region"], marker=dict(color = mapColors()), text=percent_df["Manufacturer Name"])
trace2 = go.Bar(x=percent_df["Vitebsk Region"], marker=dict(color = mapColors()), text=percent_df["Manufacturer Name"])
trace3 = go.Bar(x=percent_df["Brest Region"], marker=dict(color = mapColors()), text=percent_df["Manufacturer Name"])
trace4 = go.Bar(x=percent_df["Mogilev Region"], marker=dict(color = mapColors()), text=percent_df["Manufacturer Name"])
trace5 = go.Bar(x=percent_df["Hrodna Region"], marker=dict(color = mapColors()), text=percent_df["Manufacturer Name"])

fig_carlocs.append_trace(trace0, 1, 1)
fig_carlocs.append_trace(trace1, 1, 2)
fig_carlocs.append_trace(trace2, 2, 1)
fig_carlocs.append_trace(trace3, 2, 2)
fig_carlocs.append_trace(trace4, 3, 1)
fig_carlocs.append_trace(trace5, 3, 2)


fig_carlocs.show()

# in order on each graph from bottom to top:
# Cadillac, Acura, Buick, Mini, Infiniti, Skoda, Land Rover, Lexus, Jaguar, Porsche
# (need to find way to label/rescale graph)

# hist_carlocs = px.bar(percent_df, x="Minsk Region", orientation="h", color_discrete_map=car_colors)
# hist_carlocs.show()

In [20]:
# maxPrice_df = avgPrice_df["Manufacturer Name"].tail(10)

# cars_to_keep = []
# for car in range(10):
#     cars_to_keep.append(maxPrice_df.iloc[car])

# hist_maxCars = px.histogram(cars_df[cars_df["manufacturer_name"].isin(cars_to_keep)], x="manufacturer_name", color="location_region")
# hist_maxCars.update_layout(width=750)
# hist_maxCars.show()

# Factors that do affect the car?



In [21]:
sc_yeardrive = px.scatter(cars_df, x="year_produced", y="price_usd", facet_row="transmission", color="drivetrain",
                         labels={
                             "price_usd":"Price (USD)",
                             "year_produced":"Year Produced",
                             "drivetrain":"Drivetrain"
                         },
                         title="Year produced vs. Price")
sc_yeardrive.update_layout(height=500, width=750)
sc_yeardrive.show()

In [22]:
sc_mileage = px.scatter(cars_df, x="odometer_value", y="price_usd", color="year_produced")
sc_mileage.update_layout(height=500, width=750)
sc_mileage.show()

In [23]:
box_engine = px.box(cars_df, x="body_type", y="price_usd", color="engine_type",
                   labels={
                       "body_type":"Body Type",
                       "engine_type":"Engine Type",
                       "price_usd":"Price (USD)"
                   }, title="Body and engine type vs. Price")


box_engine.show()

- Does the price affect duration listed
- Does the manufacturer name affect the price



- Do factors that do/don't affect the car's performance \(like mileage vs. color\) affect its price?
- Are the more expensive cars all at a certain location?

